In [39]:
pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [40]:
import numpy as np
import pandas as pd

In [41]:
df=pd.read_csv('/content/diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [42]:
from sklearn.preprocessing import StandardScaler

In [43]:
X=df.iloc[:,:-1].values
y=df.iloc[:,-1].values

In [44]:
scaler=StandardScaler()
X=scaler.fit_transform(X)

In [45]:
X.shape

(768, 8)

In [46]:
from sklearn.model_selection import train_test_split

In [47]:
X_train, X_test, y_train, y_test= train_test_split(X,y,test_size=0.2, random_state=1)

In [48]:
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Dropout

In [49]:
model=Sequential()
model.add(Dense(32, activation='relu', input_shape=(8,)))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics='accuracy')

In [50]:
model.fit(X_train, y_train, batch_size=32, epochs=100, validation_data=(X_test, y_test))

Epoch 1/100
20/20 [==============================] - 1s 11ms/step - loss: 0.7921 - accuracy: 0.3143 - val_loss: 0.7332 - val_accuracy: 0.4286
Epoch 2/100
20/20 [==============================] - 0s 5ms/step - loss: 0.7275 - accuracy: 0.4414 - val_loss: 0.6769 - val_accuracy: 0.6104
Epoch 3/100
20/20 [==============================] - 0s 5ms/step - loss: 0.6750 - accuracy: 0.6075 - val_loss: 0.6351 - val_accuracy: 0.6948
Epoch 4/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6336 - accuracy: 0.6840 - val_loss: 0.5988 - val_accuracy: 0.7273
Epoch 5/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5984 - accuracy: 0.7150 - val_loss: 0.5676 - val_accuracy: 0.7468
Epoch 6/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5692 - accuracy: 0.7329 - val_loss: 0.5417 - val_accuracy: 0.7532
Epoch 7/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5461 - accuracy: 0.7329 - val_loss: 0.5233 - val_accuracy: 0.7857
Epoch 8/100


select appropriate optimizer

In [51]:
import keras_tuner as kt

In [52]:
def build_model(hp):
  model1=Sequential()
  model1.add(Dense(32, activation='relu', input_shape=(8,)))
  model1.add(Dense(1, activation='sigmoid'))
  optimizer=hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop', 'adadelta', 'adagrad'])
  model1.compile(optimizer=optimizer, loss='binary_crossentropy', metrics='accuracy')
  return model1

In [53]:
tuner= kt.RandomSearch(build_model,
                       objective='val_accuracy',
                       max_trials=10,
                       directory='tuning',
                       project_name='version1'
                       )

In [54]:
tuner.search(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

In [55]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [56]:
model2=tuner.get_best_models(num_models=1)[0]

In [57]:
model2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321
Trainable params: 321
Non-trainable params: 0
_________________________________________________________________


In [61]:
model2.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))

Epoch 1/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4533 - accuracy: 0.7834 - val_loss: 0.4619 - val_accuracy: 0.7857
Epoch 2/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4519 - accuracy: 0.7834 - val_loss: 0.4629 - val_accuracy: 0.7792
Epoch 3/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4508 - accuracy: 0.7834 - val_loss: 0.4628 - val_accuracy: 0.7792
Epoch 4/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4499 - accuracy: 0.7834 - val_loss: 0.4642 - val_accuracy: 0.7792
Epoch 5/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4493 - accuracy: 0.7883 - val_loss: 0.4658 - val_accuracy: 0.7857
Epoch 6/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4483 - accuracy: 0.7866 - val_loss: 0.4633 - val_accuracy: 0.7792
Epoch 7/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4472 - accuracy: 0.7818 - val_loss: 0.4628 - val_accuracy: 0.7857
Epoch 8/100
2

Number of neurons

In [62]:
def build_model(hp):
  model = Sequential()
  units=hp.Int('units', 8,128,1)
  model.add(Dense(
      units=units,
      activation='relu', input_shape=(8,)))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics='accuracy')
  return model

In [63]:
tuner=kt.RandomSearch(build_model,
                      objective='val_accuracy',
                      max_trials=3,
                      directory='tuning',
                      project_name='version2')

In [64]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 3 Complete [00h 00m 01s]
val_accuracy: 0.7727272510528564

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 05s


In [65]:
tuner.get_best_hyperparameters()[0].values

{'units': 80}

In [66]:
model4=tuner.get_best_models(num_models=1)[0]

In [67]:
model4.fit(X_train, y_train, batch_size=32, epochs =100, initial_epoch=5, validation_data=(X_test, y_test))

Epoch 6/100
20/20 [==============================] - 1s 11ms/step - loss: 0.4891 - accuracy: 0.7671 - val_loss: 0.4764 - val_accuracy: 0.7987
Epoch 7/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4786 - accuracy: 0.7752 - val_loss: 0.4687 - val_accuracy: 0.8052
Epoch 8/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4700 - accuracy: 0.7785 - val_loss: 0.4635 - val_accuracy: 0.8052
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4636 - accuracy: 0.7687 - val_loss: 0.4606 - val_accuracy: 0.7987
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4596 - accuracy: 0.7801 - val_loss: 0.4589 - val_accuracy: 0.7922
Epoch 11/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4563 - accuracy: 0.7785 - val_loss: 0.4568 - val_accuracy: 0.8052
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4529 - accuracy: 0.7785 - val_loss: 0.4593 - val_accuracy: 0.7857
Epoch 13/

Number of hidden layers

In [68]:
def build_model(hp):
  model=Sequential()
  model.add(Dense(72, activation='relu', input_dim=8))

  for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
    model.add(Dense(72, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics='accuracy')
  return model

In [70]:
tuner=kt.RandomSearch(build_model,
    objective='val_accuracy',
    max_trials=5,
    directory='tuning',
    project_name='version3'
)

In [71]:
tuner.search(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.8116883039474487

Best val_accuracy So Far: 0.8181818127632141
Total elapsed time: 00h 00m 14s


In [72]:
model5=tuner.get_best_hyperparameters()[0].values

In [73]:
model5

{'num_layers': 9}

In [74]:
model5=tuner.get_best_models(num_models=1)[0]

In [75]:
model5.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 72)                648       
                                                                 
 dense_1 (Dense)             (None, 72)                5256      
                                                                 
 dense_2 (Dense)             (None, 72)                5256      
                                                                 
 dense_3 (Dense)             (None, 72)                5256      
                                                                 
 dense_4 (Dense)             (None, 72)                5256      
                                                                 
 dense_5 (Dense)             (None, 72)                5256      
                                                                 
 dense_6 (Dense)             (None, 72)                5

In [76]:
model5.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))

Epoch 1/100
20/20 [==============================] - 1s 15ms/step - loss: 0.4507 - accuracy: 0.7850 - val_loss: 0.4798 - val_accuracy: 0.7922
Epoch 2/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4295 - accuracy: 0.7932 - val_loss: 0.4753 - val_accuracy: 0.7727
Epoch 3/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4151 - accuracy: 0.8127 - val_loss: 0.4649 - val_accuracy: 0.7987
Epoch 4/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4032 - accuracy: 0.8111 - val_loss: 0.4985 - val_accuracy: 0.7857
Epoch 5/100
20/20 [==============================] - 0s 6ms/step - loss: 0.3949 - accuracy: 0.8111 - val_loss: 0.4691 - val_accuracy: 0.7727
Epoch 6/100
20/20 [==============================] - 0s 6ms/step - loss: 0.3807 - accuracy: 0.8274 - val_loss: 0.5072 - val_accuracy: 0.7727
Epoch 7/100
20/20 [==============================] - 0s 5ms/step - loss: 0.3487 - accuracy: 0.8404 - val_loss: 0.5277 - val_accuracy: 0.7403
Epoch 8/100
